## Bonus-Track Assignment 5: Char RNN
Implement a Character RNN, train it to generate sequential data from your favorite author (you can also consider generating lyrics for songs). Experiment using different choices for the RNN-based language model (e.g., LSTM, GRU, RNN, etc.) and temperature for the sampling function.

A tutorial on how to start organizing your code is available at
https://colab.research.google.com/drive/1WsETcyfV7lGibKG2OojHN5AfzJmNBHT6?usp=sharing

The output of the assignment should then consist in the following
* The source code. (Please indicate the text you have used to train your model)
* The values of the hyperparameters of the model (including the temperature) used in your favorite runs
* The generated text in your favorite runs

In [30]:
import numpy as np

import torch
from tqdm import tqdm

from torch import zeros, Tensor,cuda
from torch.nn import LSTM, RNN, GRU, Linear, Module, CrossEntropyLoss
from torch.optim import AdamW
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import Dataset, DataLoader
from torch.nn.functional import softmax

In [31]:
gpu = 'cuda' if cuda.is_available() else 'cpu'

## Builder of Sentences

In [32]:
class BuildSentences:
    def __init__(self,text_file:str, max_len:int=60, step:int=3):
        """
        :param text_file: file to import
        :param max_len: Length of the extracted sequences for training
        :param step: We sample a sequence every step character
        """
        self.max_len = max_len
        sentences, next_chars = [],[]

        text = open(text_file, "r", encoding="utf-8").read().lower()

        # Build the sub-sequences
        for i in range(0, len(text) - max_len, step):
            sentences.append(text[i:i+max_len])
            next_chars.append(text[i+max_len])

        chars = sorted(list(set(text))) # List of unique characters in the corpus
        self.char2pos = {char: i for i, char in enumerate(chars)}
        self.pos2char = {i: char for i, char in enumerate(chars)}

        # Number of sentences and embedding dimension
        num_sentences, self.emb_dim = len(sentences), len(chars)

        self.x = zeros((num_sentences, max_len, self.emb_dim))
        self.y = zeros((num_sentences, self.emb_dim))

        for i, sentence in enumerate(sentences):
            for t, char in enumerate(sentence):
                self.x[i, t, self.char2pos[char]] = 1

            self.y[i,self.char2pos[next_chars[i]]] = 1


    def chars2hoe(self, chars:str)->Tensor:
        """
        Transform a string into a sequence of OHE vector, given string,
        it returns a [1, #char, emb_dim] tensor
        :param chars: Sequence of characters
        """
        hoes = zeros((len(chars), self.emb_dim))

        for idx, char in enumerate(chars):
            hoes[idx,self.char2pos[char]] = 1
        return hoes.unsqueeze(0).to(gpu)

    def hoes2chars(self, hoes:Tensor)->str:
        """
        Transform a HOE vector or a sequence of HOE vector into string
        :param hoes: HOE vector or a sequence of HOE
        """
        string = ""
        if hoes.ndim == 1: # just one character
            return self.pos2char[hoes.nonzero().item()]

        elif hoes.ndim == 2: # a sequence of characters
            for hoe in hoes:
                string += self.pos2char[hoe.nonzero().item()]

        return string

    def get_sources(self):

        return self.x , self.y

## Custom Dataset

In [33]:
class CustomDataset(Dataset):
    """
    Loaded into data-loader fot the training phase
    """
    def __init__(self, source_x:Tensor, source_y:Tensor):
        self.x, self.y = source_x, source_y

    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

## Recurrent Neural Network

In [34]:
class CharRNN(Module):
    def __init__(self,module:str, emb_dim:int, hidden:int, layers:int,bi:bool):
        super(CharRNN,self).__init__()

        if module == "RNN":
            self.RNN = RNN(emb_dim, hidden, layers, bidirectional=bi, batch_first=True)
        elif module == "LSTM":
            self.RNN = LSTM(emb_dim, hidden, layers, bidirectional=bi,batch_first=True)
        elif module == "GRU":
            self.RNN = GRU(emb_dim, hidden, layers, bidirectional=bi, batch_first=True)

        B = 2 if bi else 1
        self.readout = Linear(B * hidden, emb_dim)
        self.criteria = CrossEntropyLoss()

    def forward(self,seq:Tensor, y:Tensor=None):

        out, _ = self.RNN(seq)
        out =  self.readout(out[:,-1, :]) # take the last hidden step

        # Perform the loss if possible
        loss = None
        if y is not None:
            loss = self.criteria(out, y)

        # perform the softmax
        out = softmax(out.detach(), -1)
        return (out, loss) if loss is not None else out

## Training function

In [35]:
def sample(logits:Tensor, temperature =1.0):
    """
    Softmax with temperature
    :param logits: output of the model (probability distribution)
    :param temperature: temperature scaling
    """
    logits = np.asarray(logits.cpu().numpy()).astype('float64')
    logits = np.log(logits)/temperature
    exp_logits = np.exp(logits)
    soft_max = exp_logits / np.sum(exp_logits)
    return np.random.multinomial(1, soft_max, 1).argmax()

In [36]:
def train(model:Module, dt:Dataset, epoch:int, lr:float):

    optimizer = AdamW(model.parameters(),lr)
    loader = DataLoader(dt,batch_size=512, shuffle=True)
    loss = 0

    for i in tqdm(range(epoch)):
        for x, y in loader:
            x, y = x.to("cuda"), y.to("cuda")

            optimizer.zero_grad(set_to_none=True)
            _, loss = model(x, y)
            loss.backward()
            clip_grad_norm_(model.parameters(), 1)
            optimizer.step()
        if i % 80 == 0:
            print(f"Epoch {i} loss {round(loss.item(), 5)}")


In [37]:
def predict_song(model:Module,t:float, input_seq:str, length:int, handler:BuildSentences):

    input_seq = handler.chars2hoe(input_seq.lower()).to("cuda")

    model_rnn.eval()
    with torch.no_grad():
        for i in range(length):

            pred_out = model(input_seq)[0]
            char_out = handler.pos2char[sample(pred_out, temperature=t)]
            char_out_hoe = handler.chars2hoe(char_out)
            input_seq = torch.cat((input_seq, char_out_hoe), dim=1)[:,-handler.emb_dim:]
            print(char_out, end="")


In [38]:
builder = BuildSentences("sources/song.txt")
custom_dt = CustomDataset(*builder.get_sources())

input_sequence = "Più di un film, più di un drink, più della marijuana!"
hidden_dim =  128
hidden_layers = 2
learning_rate = 0.005
max_epochs = 150
bidirectional = True

## Experiements

### RNN

In [39]:
model_rnn = CharRNN("RNN", builder.emb_dim, hidden_dim, hidden_layers, bidirectional).to(gpu)
train(model_rnn, custom_dt, max_epochs, learning_rate)

  1%|▏         | 2/150 [00:00<00:28,  5.26it/s]

Epoch 0 loss 3.1217


 54%|█████▍    | 81/150 [00:04<00:03, 21.12it/s]

Epoch 80 loss 0.0445


100%|██████████| 150/150 [00:07<00:00, 19.85it/s]


#### Varying the temperature

In [40]:
for temp in [0.2, 0.5, 1.0, 1.2]:
    print(f"\n----Temperature {temp} ----")
    predict_song(model_rnn, temp, input_sequence, 400, builder)
    print("")


----Temperature 0.2 ----

più della miesta co alta
pa!
più di un frip di sta miss mezza brasiliana
dita
di un trip di sta miss mezza brasiliana
dita
di un trip di sta miss mezza brasiliana
dita
di un trip di sta miss mezza brasiliana
dita
di un trip di sta miss mezza brasiliana
dita
di un trip di sta miss mezza brasiliana
dita
di un trip di sta miss mezza brasiliana
dita, niù della coca
parnana
più di un frip di sta miss 

----Temperature 0.5 ----

più della coca
parna
più di un frip di sta miss mezza brasiliana
dita
di un trip di sta miss mezza brasiliana
dita
di un trip di sta miss mezza brasiliana
dita
di un trip di sta miss mezza brasiliana!
di un fertoran arca
più mell’o o alla savonana
più di un frip di sta miss mezza brasiliana
dita
di un trip di sta miss mezza brasiliana
dita
di un trip di sta miss mezza brasiliana
dita
di un trip d

----Temperature 1.0 ----

più della coca
parnana!
più della mersa an na batarta!
più di un fuelta, pie della coca
parta
più dello mo sn un ta tag

### LSTM

In [41]:
model_rnn = CharRNN("LSTM", builder.emb_dim, hidden_dim, hidden_layers, bidirectional).to(gpu)
train(model_rnn, custom_dt, max_epochs, learning_rate)

  1%|▏         | 2/150 [00:00<00:19,  7.43it/s]

Epoch 0 loss 3.49963


 55%|█████▍    | 82/150 [00:11<00:09,  6.97it/s]

Epoch 80 loss 0.55416


100%|██████████| 150/150 [00:21<00:00,  7.14it/s]


#### Varying the temperature

In [42]:
for temp in [0.2, 0.5, 1.0, 1.2]:
    print(f"\n----Temperature {temp} ----")
    predict_song(model_rnn, temp, input_sequence, 400, builder)
    print("")


----Temperature 0.2 ----

più della suita copo ball della ho aprpiù della suita copo ball’a, diammi un brivonona!
di un trip di sta miss mezza brasiliana!
di un trip di sta miss mezza brasiliana!
di un trip di sta miss mezza brasiliana!
di un trip di sta miss mezza brasiliana!
di un trip di sta miss mezza brasiliana!
di un trip di sta miss mezza brasiliana!
di un trip di sta miss mezza brasiliana!
di un triso briù di ogni

----Temperature 0.5 ----

più dellla coca
più dei giielli edessta sita!
più di un trip di sta miss mezza brasiliana!
di un trip di sta miss mezza brasiliana!
di un trip di sta miss mezza brasiliana!
di un trip di sta miss mezza brasiliana!
di un trip di sta miss mezza brasiliana!
di un trip di sta miss mezza brasiliana!
di un trip di sta miss mezza brasiliana!
di un tritognie più di ogni banconota, più della coca
più dei 

----Temperature 1.0 ----

più di un trip di sta miss mezza brasiliana!
di un to iun ove drita!
più di un trip di sta mistuo sue mozza!
più di un 

### GRU

In [43]:
model_rnn = CharRNN("GRU", builder.emb_dim, hidden_dim, hidden_layers, bidirectional).to(gpu)
train(model_rnn, custom_dt, max_epochs, learning_rate)

  1%|▏         | 2/150 [00:00<00:13, 11.25it/s]

Epoch 0 loss 3.20066


 55%|█████▍    | 82/150 [00:07<00:06, 10.48it/s]

Epoch 80 loss 0.01674


100%|██████████| 150/150 [00:14<00:00, 10.46it/s]


#### Varying the temperature

In [44]:
for temp in [0.2, 0.5, 1.0, 1.2]:
    print(f"\n----Temperature {temp} ----")
    predict_song(model_rnn, temp, input_sequence, 400, builder)
    print("")


----Temperature 0.2 ----

più di un trip di sta miss mezza brasiliana!
dita missa con he più quello che ho prozzara!
più della missa mila si abana!
più della coca
sanana!
più di un trip di sta miss mezza brasiliana!
dita missa mita!
si amezzababrilizana!
più di un trip di sta miss mezzabana!
più della coca
sanana!
più di un trip di sta miss mezza brasiliana!
dita missa con he più quello che ho provei io cono!
famminan bri

----Temperature 0.5 ----

più di un trip di sta miss mezza brasiliana!
did di sun furta
più della misagia!
miù di un trip di sta miss mezza brasiliana!
dita più di ogni banconota, più della coca
sanana!
più di un trip di sta miss mezza brasiliana!
dita missa co ho proveie in cochota, più della coca
sanana!
più di un trip di sta miss mezzabana!
più della coca
più deila coca
più deil si eilo co ho ta trita!
si amelli cove!


----Temperature 1.0 ----

più di un trip di sta miss mezza brasiliana!
dita picabana!
più dello comedoka!
più di un trip di sta miss mezza brasil